# 부록 10.2.4: 도구 선택

Claude API는 `tool_choice` 라는 매개변수를 지원하여 Claude가 도구를 호출하는 방식을 지정할 수 있습니다. 이 노트북에서는 이 매개변수가 어떻게 작동하는지와 언제 사용해야 하는지 살펴보겠습니다.

`tool_choice` 매개변수를 사용할 때 세 가지 옵션이 있습니다: 

* `auto`는 Claude가 제공된 도구를 호출할지 여부를 결정하도록 허용합니다.
* `any`는 Claude에게 제공된 도구 중 하나를 사용하도록 지시하지만 특정 도구를 강제하지는 않습니다.
* `tool`은 Claude에게 특정 도구를 항상 사용하도록 강제합니다.


다음 다이어그램은 각 옵션이 어떻게 작동하는지 보여줍니다: 

![tool_choice.png](./images/tool_choice.png)

각 옵션을 자세히 살펴보겠습니다. Anthropic SDK를 가져오는 것부터 시작하겠습니다:

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt

In [ ]:
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

In [ ]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print(f'Using region: ', {region})

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

## 자동

`tool_choice`를 `auto`로 설정하면 모델이 도구를 사용할지 여부를 자동으로 결정할 수 있습니다. 이것이 `tool_choice` 매개변수를 사용하지 않을 때 도구를 사용할 때의 기본 동작입니다.

이를 보여주기 위해 Claude에게 가짜 웹 검색 도구를 제공할 것입니다. Claude에게 질문을 하는데, 그중 일부는 웹 검색 도구를 호출해야 하고 다른 일부는 Claude가 스스로 답할 수 있어야 합니다.

먼저 `web_search`라는 도구를 정의합시다. 이 데모를 간단하게 유지하기 위해 실제로 웹을 검색하지는 않습니다.

`toolChoice`를 `auto`로 설정합니다.

In [ ]:
def web_search(topic):
    print(f"pretending to search the web for {topic}")

toolConfig = {'tools': [],
        "toolChoice": {
        "auto":{},
    }
}

toolConfig['tools'].append({
      "toolSpec": {
        "name": "web_search",
        "description": "A tool to retrieve up to date information on a given topic by searching the web. Only search the web for queries that you can not confidently answer.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "topic": {"type": "string", "description": "The topic to search the web for"}
            },
            "required": ["topic"]
          }
        }
      }
    })

다음으로 `user_query`를 받아 Claude에 전달하고 `web_search_tool`도 함께 전달하는 함수를 작성합니다. 

전체 함수는 다음과 같습니다:

In [ ]:
from datetime import date

def chat_with_web_search(user_query):
    messages = [{"role": "user", "content": [{"text": user_query}]}]

    system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.  
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
    """

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig
    }

    response = bedrock_client.converse(**converse_api_params)

    stop_reason = response['stopReason']

    if stop_reason == "end_turn":
        print("Claude did NOT call a tool")
        print(f"Assistant: {stop_reason}")
    elif stop_reason == "tool_use":
        print("Claude wants to use a tool")
        print(stop_reason)

Claude가 도구를 사용하지 않고 답할 수 있는 질문으로 시작해 보겠습니다:

In [ ]:
chat_with_web_search("What color is the sky?")

"하늘은 무슨 색인가요?"라고 물었을 때, Claude는 도구를 사용하지 않습니다. Claude가 웹 검색 도구를 사용해야 하는 질문을 해 보겠습니다:

In [ ]:
chat_with_web_search("Who won the 2024 Miami Grand Prix?")

"2024 마이애미 그랑프리에서 누가 우승했나요?"라고 물었을 때, Claude는 웹 검색 도구를 사용합니다! 

몇 가지 예시를 더 살펴보겠습니다:

In [ ]:
# Claude should NOT need to use the tool for this:
chat_with_web_search("Who won the Superbowl in 2022?")

In [ ]:
# Claude SHOULD use the tool for this:
chat_with_web_search("Who won the Superbowl in 2024?")

### 프롬프트가 중요합니다!

`toolChoice`를 `auto`로 설정할 때, 상세한 프롬프트를 작성하는 것이 중요합니다. 종종 Claude는 도구를 호출하는 데 너무 적극적입니다. 상세한 프롬프트를 작성하면 Claude가 도구를 호출해야 하는지 여부를 판단하는 데 도움이 됩니다. 위의 예시에서 우리는 시스템 프롬프트에 구체적인 지침을 포함시켰습니다:

In [ ]:
system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
"""

***

## 특정 도구 강제 사용

`toolChoice`를 사용하여 Claude에게 특정 도구를 사용하도록 강제할 수 있습니다. 아래 예시에서는 두 가지 간단한 도구를 정의했습니다: 
* `print_sentiment_scores` - Claude를 "속여" 감정 분석 데이터를 포함하는 잘 구조화된 JSON 출력을 생성하도록 하는 도구. 이 접근 방식에 대한 자세한 내용은 Anthropic Cookbook의 [Extracting Structured JSON using Claude and Tool Use](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/extracting_structured_json.ipynb)를 참조하세요.
* `calculator` - 두 개의 숫자를 더하는 매우 간단한 계산기 도구입니다.


우리의 목표는 `analyze_tweet_sentiment`라는 함수를 작성하는 것입니다. 이 함수는 트윗을 입력받아 Claude를 사용하여 해당 트윗에 대한 기본적인 감정 분석을 출력합니다. 결국에는 Claude가 `print_sentiment_scores` 도구를 사용하도록 "강제"할 것입니다. 하지만 우선 도구 사용을 **강제하지 않았을 때** 어떤 일이 일어나는지 보여드리겠습니다.이 첫 번째 "나쁜" 버전의 `analyze_tweet_sentiment` 함수에서는 Claude에게 두 가지 도구를 모두 제공합니다. 비교를 위해 `toolChoice`를 `auto`로 설정하겠습니다.

In [ ]:
# Create our toolConfig
toolConfig = {'tools': [],
        "toolChoice": {
        "auto":{},
    }
}

# append our print_sentiment_scores tool
toolConfig['tools'].append({
    "toolSpec": {
      "name": "print_sentiment_scores",
      "description": "Prints the sentiment scores of a given tweet or piece of text.",
      "inputSchema": {
        "json": {
          "type": "object",
          "properties": {
            "positive_score": {"type": "number","description": "The positive sentiment score, ranging from 0.0 to 1.0."},
            "negative_score": {"type": "number","description": "The negative sentiment score, ranging from 0.0 to 1.0."},
            "neutral_score": {"type": "number","description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
          },
          "required": ["positive_score", "negative_score", "neutral_score"]
        }
      }
    }
  })

# Append our Calculator tool
toolConfig['tools'].append({
    "toolSpec": {
      "name": "calculator",
      "description": "Adds two numbers",
      "inputSchema": {
        "json": {
          "type": "object",
          "properties": {
            "num1": {"type": "number", "description": "first number to add"},
            "num2": {"type": "number", "description": "second number to add"}
          },
          "required": ["num1", "num2"]
        }
      }
    }
  })

Claude에게 잘 작성된 프롬프트를 제공하지 않는 것에 유의하시기 바랍니다. 이렇게 하면 특정 도구 사용을 강제했을 때의 영향을 더 쉽게 볼 수 있습니다.

In [ ]:
def analyze_tweet_sentiment(query):
    messages = [{"role": "user", "content": [{"text": query}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig,
    }
    response = bedrock_client.converse(**converse_api_params)
    print(response)

함수를 호출할 때 트윗 `Holy cow, I just made the most incredible meal!`을 사용해 보겠습니다.

In [ ]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

Claude는 `print_sentiment_scores` 도구를 호출하지 않고 직접 응답합니다.\n> "That's great to hear! I don't actually have the capability to assess sentiment from text, but it sounds like you're really excited and proud of the incredible meal you made\n\n다음으로 누군가가 이렇게 트윗했다고 가정해 봅시다. `I love my cats! I had four and just adopted 2 more! Guess how many I have now?`

In [ ]:
analyze_tweet_sentiment("I love my cats! I had four and just adopted 2 more! Guess how many I have now?")

Claude는 계산기 도구를 호출하려고 합니다.\n\n> {'toolUse': {'toolUseId': 'tooluse_oyzX9vToT468sAwe_A99EA', **'name': 'calculator', 'input': {'num1': 4, 'num2': 2}**}}]}}, 'stopReason': 'tool_use'{'toolUse': {'toolUseId': 'tooluse_oyzX9vToT468sAwe_A99EA', 'name': 'calculator', 'input': {'num1': 4, 'num2': 2}}}]}}, 'stopReason': 'tool_use'

명백히 현재 구현 방식은 우리가 원하는 대로 작동하지 않습니다(대부분 실패하도록 설정했기 때문입니다).그래서 `toolChoice`를 업데이트하여 Claude가 **항상** `print_sentiment_scores` 도구를 사용하도록 강제하겠습니다.

In [ ]:
toolConfig['toolChoice'] = {
    "tool": {
        "name": "print_sentiment_scores"}
}

또한 `type`을 `tool`로 설정하고 특정 도구 이름을 제공해야 합니다.

In [ ]:
def analyze_tweet_sentiment(query):
    messages = [{"role": "user", "content": [{"text": query}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig,
    }
    response = bedrock_client.converse(**converse_api_params)
    print(response)

이제 앞서와 같은 프롬프트로 Claude에게 입력하면 항상 `print_sentiment_scores` 도구를 호출할 것입니다.

In [ ]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

Claude는 `print_sentiment_scores` 도구를 호출합니다.\n\n> [{'toolUse': {'toolUseId': 'tooluse_EZnn27PHRXWfo7JR8FWkDw', **'name': 'print_sentiment_scores',** 'input': {'positive_score': 0.9, 'negative_score': 0.1, 'neutral_score': 0.0}}}][{'toolUse': {'toolUseId': 'tooluse_EZnn27PHRXWfo7JR8FWkDw', 'name': 'print_sentiment_scores', 'input': {'positive_score': 0.9, 'negative_score': 0.1, 'neutral_score': 0.0}}}]\n\n"수학적인" 트윗으로 Claude를 속이려고 해도 여전히 `print_sentiment_scores` 도구를 호출합니다.

In [ ]:
analyze_tweet_sentiment("I love my cats! I had four and just adopted 2 more! Guess how many I have now?")

Claude가 `print_sentiment_scores` 도구를 호출하도록 강제하더라도 기본적인 프롬프트 엔지니어링을 사용하여 Claude에게 작업 컨텍스트를 더 잘 제공해야 합니다.

In [ ]:
def analyze_tweet_sentiment(query):
    prompt = f"""
    Analyze the sentiment in the following tweet:
    <tweet>{query}</tweet>"""

    messages = [{"role": "user", "content": [{"text": prompt}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig,
    }
    response = bedrock_client.converse(**converse_api_params)
    print(response)

***

## 아무 도구

`toolChoice`의 마지막 옵션은 `any`로, 이를 통해 Claude에게 "도구를 호출해야 하지만 어떤 도구를 선택할지는 당신이 결정하라"고 지시할 수 있습니다. Claude를 사용하여 SMS 채팅봇을 만들고 싶다고 가정해 봅시다. 이 채팅봇이 사용자와 실제로 "대화"할 수 있는 유일한 방법은 SMS 텍스트 메시지를 통해서입니다.

아래 예시에서 우리는 두 가지 도구에 액세스할 수 있는 매우 단순한 텍스트 메시징 어시스턴트를 만듭니다:
* `send_text_to_user` - 사용자에게 텍스트 메시지를 보냅니다.
* `get_customer_info` - 사용자 이름을 기반으로 고객 데이터를 조회합니다.

여기서 아이디어는 항상 이 두 도구 중 하나를 호출하고 도구 응답이 아닌 다른 응답을 하지 않는 채팅봇을 만드는 것입니다. 모든 상황에서 Claude는 텍스트 메시지를 보내거나 `get_customer_info`를 호출하여 더 많은 고객 정보를 얻어야 합니다. 이를 보장하기 위해 `toolChoice`를 `any`로 설정합니다:

In [ ]:
toolConfig = {'tools': [],
        "toolChoice": {
        "any":{},
    }
}

toolConfig['tools'].append({
      "toolSpec": {
        "name": "send_text_to_user",
        "description": "Sends a text message to a user",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "text": {
                "type": "string",
                "description": "The piece of text to be sent to the user via text message"}
            },
            "required": ["text"]
          }
        }
      }
    })

toolConfig['tools'].append({
      "toolSpec": {
        "name": "get_customer_info",
        "description": "gets information on a customer based on the customer's username.  Response includes email, username, and previous purchases. Only call this tool once a user has provided you with their username",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "username": {
                "type": "string",
                "description": "The username of the user in question. "}
            },
            "required": ["username"]
          }
        }
      }
    })

In [ ]:
#toolConfig # Optional uncomment to see the updated toolConfig

In [ ]:
def send_text_to_user(text):
    # Sends a text to the user
    # We'll just print out the text to keep things simple:
    print(f"TEXT MESSAGE SENT: {text}")

def get_customer_info(username):
    return {
        "username": username,
        "email": f"{username}@email.com",
        "purchases": [
            {"id": 1, "product": "computer mouse"},
            {"id": 2, "product": "screen protector"},
            {"id": 3, "product": "usb charging cable"},
        ]
    }

system_prompt = """
All your communication with a user is done via text message.
Only call tools when you have enough information to accurately call them.  
Do not call the get_customer_info tool until a user has provided you with their username. This is important.
If you do not know a user's username, simply ask a user for their username.
"""

def sms_chatbot(user_message):
    messages = [{"role": "user", "content": [{"text": user_message}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": system_prompt}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 1000},
        "toolConfig":toolConfig,
    }

    response = bedrock_client.converse(**converse_api_params)

    if(response['stopReason'] == "tool_use"):
        tool_use = response['output']['message']['content'][-1]
        tool_name = tool_use['toolUse']['name']
        tool_inputs = tool_use['toolUse']['input']
        print(f"=======Claude Wants To Call The {tool_name} Tool=======")
        if tool_name == "send_text_to_user":
            send_text_to_user(tool_inputs["text"])
        elif tool_name == "get_customer_info":
            print(get_customer_info(tool_inputs["username"]))
        else:
            print("Oh dear, that tool doesn't exist!")
            
    else:
        print("No tool was called. This shouldn't happen!")

간단하게 시작해 봅시다:

In [ ]:
sms_chatbot("Hey there! How are you?")

Claude는 `send_text_to_user` 도구를 호출하여 응답합니다.

다음으로 Claude에게 조금 더 까다로운 질문을 해 봅시다:

In [ ]:
sms_chatbot("I need help looking up an order")

Claude는 사용자에게 사용자 이름을 제공하라는 텍스트 메시지를 보내고 싶어 합니다.

이제 Claude에게 우리의 사용자 이름을 제공했을 때 어떤 일이 일어나는지 봅시다:

In [ ]:
sms_chatbot("I need help looking up an order.  My username is jenny76")

Claude는 우리가 기대한 대로 `get_customer_info` 도구를 호출합니다!

우리가 Claude에게 무의미한 메시지를 보내더라도 여전히 우리의 도구 중 하나를 호출할 것입니다:

In [ ]:
sms_chatbot("askdj aksjdh asjkdbhas kjdhas 1+1 ajsdh")